In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('C:/path/to/the/file/tripadvisor_prague_restaurants.csv')

In [3]:
# Drop these columns
columns_to_drop = [
    'original_location', 'country', 'region', 'province', 'city', 'top_tags', 'price_level',
    'special_diets', 'default_language', 'popularity_detailed', 'working_shifts_per_week',
    'reviews_count_in_default_language', 'excellent', 'very_good', 'average', 'poor', 'terrible'
]

df = df.drop(columns=columns_to_drop)

In [4]:
# Drop duplicates if there are any
df = df.drop_duplicates()

In [5]:
# Rename the column 'restaurant_link'
df = df.rename(columns={'restaurant_link': 'tripadvisor_link'})

In [6]:
# Sort the DataFrame by 'restaurant_name' alphabetically 
df = df.sort_values(by='restaurant_name', ascending=True)

In [7]:
# Replace null values in location columns
df['latitude'].fillna(0, inplace=True)
df['longitude'].fillna(0, inplace=True)

In [8]:
# Remove everything after the first comma in the address column (keep only street)
df['address'] = df['address'].apply(lambda x: x.split(',')[0])

In [9]:
# Convert 'claimed' column to boolean datatype
df['claimed'] = df['claimed'].apply(lambda x: True if x == 'Claimed' else False)

In [10]:
# Convert 'awards' column strings to lists of strings, keeping NaN values
df['awards'] = df['awards'].apply(
    lambda x: [award.strip() for award in x.split(',')] if pd.notna(x) and isinstance(x, str) else x
)

In [11]:
# Convert 'meals' column strings to lists of strings, keeping NaN values
df['meals'] = df['meals'].apply(lambda x: [meal.strip() for meal in x.split(',')] if pd.notna(x) else x)

In [12]:
# Convert 'cuisines' column strings to lists of strings, keeping NaN values
df['cuisines'] = df['cuisines'].apply(lambda x: [cuisine.strip() for cuisine in x.split(',')] if pd.notna(x) else x)

In [13]:
# Convert 'features' column strings to lists of strings, keeping NaN values
df['features'] = df['features'].apply(lambda x: [feature.strip() for feature in x.split(',')] if pd.notna(x) else x)

In [14]:
# Convert 'keywords' column strings to lists of strings, keeping NaN values
df['keywords'] = df['keywords'].apply(lambda x: [keyword.strip() for keyword in x.split(',')] if pd.notna(x) else x)

In [15]:
# Extract the rank as an integer from 'popularity_generic', preserving NaN values
df['popularity_rnk_all_restaurants'] = df['popularity_generic'].str.extract(r'#(\d+)').astype(float)  # Extract as float to handle NaNs
df['popularity_rnk_all_restaurants'] = df['popularity_rnk_all_restaurants'].astype('Int64')  # Convert to Int64 to preserve NaNs
# Drop the old 'popularity_generic' column
df = df.drop(columns=['popularity_generic'])

In [16]:
# Extract 'eur_price_range_from' and 'eur_price_range_to'
df[['eur_price_range_from', 'eur_price_range_to']] = df['price_range'].str.extract(r'€([\d,]+)-€([\d,]+)')
# Remove commas and convert the extracted columns to float, keeping NaN values intact
df['eur_price_range_from'] = df['eur_price_range_from'].str.replace(',', '').astype(float)
df['eur_price_range_to'] = df['eur_price_range_to'].str.replace(',', '').astype(float)
# Convert to integers while keeping NaN values
df['eur_price_range_from'] = df['eur_price_range_from'].astype('Int64')
df['eur_price_range_to'] = df['eur_price_range_to'].astype('Int64')
# Drop the original 'price_range' column
df.drop(columns=['price_range'], inplace=True)

In [17]:
# Convert 'open_days_per_week', 'open_hours_per_week' and 'total_reviews_count' to integers while preserving NaN values
df['open_days_per_week'] = df['open_days_per_week'].apply(lambda x: int(x) if pd.notna(x) else pd.NA)
df['open_hours_per_week'] = df['open_hours_per_week'].apply(lambda x: int(x) if pd.notna(x) else pd.NA)
df['total_reviews_count'] = df['total_reviews_count'].apply(lambda x: int(x) if pd.notna(x) else pd.NA)

In [18]:
# Convert 'Y'/'N' to boolean values
df['vegetarian_friendly'] = df['vegetarian_friendly'].map({'Y': True, 'N': False})
df['vegan_options'] = df['vegan_options'].map({'Y': True, 'N': False})
df['gluten_free'] = df['gluten_free'].map({'Y': True, 'N': False})

In [19]:
# Reset index and set it as a column named 'restaurant_id'
df.reset_index(drop=True, inplace=True) 
df['restaurant_id'] = df.index + 1

In [20]:
# Reorder the columns
order = [
    'restaurant_id', 'restaurant_name', 'address', 'longitude', 'latitude', 'claimed',
    'avg_rating', 'total_reviews_count', 'popularity_rnk_all_restaurants', 'awards',
    'eur_price_range_from', 'eur_price_range_to', 'meals', 'cuisines', 'features',
    'vegetarian_friendly', 'vegan_options', 'gluten_free', 'open_days_per_week',
    'open_hours_per_week', 'original_open_hours', 'food', 'service', 'value',
    'atmosphere', 'tripadvisor_link', 'keywords'
]
df = df.reindex(columns=order)

In [21]:
# Create restaurant_dim table
restaurant_dim_columns = [
    'restaurant_id', 'restaurant_name', 'claimed', 'awards', 'tripadvisor_link',
    'keywords', 'open_days_per_week', 'open_hours_per_week', 'original_open_hours'
]
restaurant_dim = df[restaurant_dim_columns]
restaurant_dim

,restaurant_id,restaurant_name,claimed,awards,tripadvisor_link,keywords,open_days_per_week,open_hours_per_week,original_open_hours
0,1,"0,75 [Sedmička]",True,NaN,g274707-d21335733,NaN,<NA>,<NA>,NaN
1,2,1961 Espresso Bar,False,NaN,g274707-d21268091,NaN,<NA>,<NA>,NaN
2,3,2 Steps 2 Heaven Café,True,NaN,g274707-d15115433,NaN,5,48,"{""Mon"": [], ""Tue"": [""09:00-19:00""], ""Wed"": [""0..."
3,4,2+KK Restaurant,True,"[Travellers' Choice, Certificate of Excellence...",g274707-d5602435,NaN,7,91,"{""Mon"": [""11:00-00:00""], ""Tue"": [""11:00-00:00""..."
4,5,2002 Beer & Kitchen,False,NaN,g274707-d783306,"[ribs, duck, mashed potatoes, goulash, old town]",7,106,"{""Mon"": [""08:00-00:00""], ""Tue"": [""08:00-00:00""..."
...,...,...,...,...,...,...,...,...,...
6030,6031,Štěrba,False,NaN,g274707-d15218975,NaN,<NA>,<NA>,NaN
6031,6032,Švejk Restaurant,False,NaN,g274707-d15291163,NaN,7,167,"{""Mon"": [""00:00-23:59""], ""Tue"": [""00:00-23:59""..."
6032,6033,Živá Kavárna,False,NaN,g274707-d17577546,NaN,<NA>,<NA>,NaN
6033,6034,Žižkovská štrúdlárna,False,NaN,g274707-d2193858,NaN,5,20,"{""Mon"": [""13:00-17:00""], ""Tue"": [""13:00-17:00""..."


In [22]:
# Create location_dim table
location_dim_columns = ['restaurant_id', 'address', 'longitude', 'latitude']
location_dim = df[location_dim_columns].drop_duplicates().reset_index(drop=True)
location_dim['location_id'] = range(1, len(location_dim) + 1)
location_dim = location_dim[['location_id', 'restaurant_id', 'address', 'longitude', 'latitude']]
location_dim

,location_id,restaurant_id,address,longitude,latitude
0,1,1,Dukelských Hrdinů 975/14,14.433153,50.098650
1,2,2,Na Příkopě 10,14.425698,50.084496
2,3,3,Ondříčkova 1244/15,14.448580,50.078960
3,4,4,Namesti Miru 342/13,14.437007,50.074600
4,5,5,Hybernska 7/1033,14.431410,50.087368
...,...,...,...,...,...
6030,6031,6031,Ústřední 289/26a,14.553446,50.071970
6031,6032,6032,Průběžná 1174/28,14.491560,50.068760
6032,6033,6033,Výpadová 1676/4a Praha - Radotín,14.366340,49.983270
6033,6034,6034,Jeseniova 909/29,14.466810,50.088020


In [23]:
# Create price_range_dim table
price_range_dim_columns = ['restaurant_id', 'eur_price_range_from', 'eur_price_range_to']
price_range_dim = df[price_range_dim_columns].drop_duplicates().reset_index(drop=True)
price_range_dim['price_range_id'] = range(1, len(price_range_dim) + 1)
price_range_dim = price_range_dim[['price_range_id', 'restaurant_id', 'eur_price_range_from', 'eur_price_range_to']]
price_range_dim

,price_range_id,restaurant_id,eur_price_range_from,eur_price_range_to
0,1,1,6,10
1,2,2,1,5
2,3,3,1,6
3,4,4,7,20
4,5,5,<NA>,<NA>
...,...,...,...,...
6030,6031,6031,<NA>,<NA>
6031,6032,6032,<NA>,<NA>
6032,6033,6033,<NA>,<NA>
6033,6034,6034,<NA>,<NA>


In [24]:
# Convert list-like columns to tuples (to handle error from drop_duplicates())
df['meals'] = df['meals'].apply(lambda x: tuple(x) if isinstance(x, list) else x)
df['cuisines'] = df['cuisines'].apply(lambda x: tuple(x) if isinstance(x, list) else x)
df['features'] = df['features'].apply(lambda x: tuple(x) if isinstance(x, list) else x)

# Create attributes_dim table
attributes_dim_columns = ['restaurant_id', 'meals', 'cuisines', 'features']
attributes_dim = df[attributes_dim_columns].drop_duplicates().reset_index(drop=True)
attributes_dim['attributes_id'] = range(1, len(attributes_dim) + 1)

# Convert tuple columns back to lists and reorder
attributes_dim['meals'] = attributes_dim['meals'].apply(lambda x: list(x) if isinstance(x, tuple) else x)
attributes_dim['cuisines'] = attributes_dim['cuisines'].apply(lambda x: list(x) if isinstance(x, tuple) else x)
attributes_dim['features'] = attributes_dim['features'].apply(lambda x: list(x) if isinstance(x, tuple) else x)
attributes_dim = attributes_dim[['attributes_id', 'restaurant_id', 'meals', 'cuisines', 'features']]
attributes_dim

,attributes_id,restaurant_id,meals,cuisines,features
0,1,1,NaN,[Wine Bar],NaN
1,2,2,NaN,[Cafe],NaN
2,3,3,"[Breakfast, Drinks]","[French, European, Czech]",NaN
3,4,4,"[Lunch, Dinner]","[Bar, European, Czech, Eastern European, Centr...","[Reservations, Seating, Serves Alcohol, Full B..."
4,5,5,"[Lunch, Dinner, Brunch, After-hours]","[Pub, Czech]","[Takeout, Reservations, Outdoor Seating, Seati..."
...,...,...,...,...,...
6030,6031,6031,NaN,"[European, Czech]",NaN
6031,6032,6032,"[Breakfast, Lunch, Dinner]",NaN,NaN
6032,6033,6033,NaN,[Healthy],NaN
6033,6034,6034,NaN,[European],[Takeout]


In [25]:
# Create dietary_dim table
dietary_dim_columns = ['restaurant_id', 'vegetarian_friendly', 'vegan_options', 'gluten_free']
dietary_dim = df[dietary_dim_columns].drop_duplicates().reset_index(drop=True)
dietary_dim['dietary_id'] = range(1, len(dietary_dim) + 1)
dietary_dim = dietary_dim[['dietary_id', 'restaurant_id', 'vegetarian_friendly', 'vegan_options', 'gluten_free']]
dietary_dim

,dietary_id,restaurant_id,vegetarian_friendly,vegan_options,gluten_free
0,1,1,False,False,False
1,2,2,False,False,False
2,3,3,False,False,False
3,4,4,True,False,False
4,5,5,False,False,False
...,...,...,...,...,...
6030,6031,6031,False,False,False
6031,6032,6032,False,False,False
6032,6033,6033,False,False,False
6033,6034,6034,False,False,False


In [26]:
# Create restaurant_reviews_fact table
restaurant_reviews_fact = df[['restaurant_id', 'avg_rating', 'total_reviews_count', 'popularity_rnk_all_restaurants',
                              'food', 'service', 'value', 'atmosphere']]

# Merge with other tables to get FKs
restaurant_reviews_fact = restaurant_reviews_fact.merge(location_dim[['restaurant_id', 'location_id']],
                                                        on='restaurant_id', how='left')

restaurant_reviews_fact = restaurant_reviews_fact.merge(price_range_dim[['restaurant_id', 'price_range_id']],
                                                        on='restaurant_id', how='left')

restaurant_reviews_fact = restaurant_reviews_fact.merge(dietary_dim[['restaurant_id', 'dietary_id']],
                                                        on='restaurant_id', how='left')

restaurant_reviews_fact = restaurant_reviews_fact.merge(attributes_dim[['restaurant_id', 'attributes_id']],
                                                        on='restaurant_id', how='left')

# Assign review_id as a unique identifier for each row (PK)
restaurant_reviews_fact['review_id'] = range(1, len(restaurant_reviews_fact) + 1)

# Reorder the columns
restaurant_reviews_fact = restaurant_reviews_fact[['review_id', 'restaurant_id', 'location_id', 'price_range_id', 
                                                   'attributes_id', 'dietary_id', 'avg_rating', 'total_reviews_count', 
                                                   'popularity_rnk_all_restaurants', 'food', 'service', 'value', 
                                                   'atmosphere']]

# Ensure no duplicate rows exist in the fact table
restaurant_reviews_fact = restaurant_reviews_fact.drop_duplicates().reset_index(drop=True)


restaurant_reviews_fact

,review_id,restaurant_id,location_id,price_range_id,attributes_id,dietary_id,avg_rating,total_reviews_count,popularity_rnk_all_restaurants,food,service,value,atmosphere
0,1,1,1,1,1,1,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN
1,2,2,2,2,2,2,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN
2,3,3,3,3,3,3,4.5,4,3207,NaN,NaN,NaN,NaN
3,4,4,4,4,4,4,4.5,201,505,4.5,4.5,4.5,4.5
4,5,5,5,5,5,5,3.5,1077,1075,4.0,3.5,4.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6030,6031,6031,6031,6031,6031,6031,4.0,10,2393,NaN,NaN,NaN,NaN
6031,6032,6032,6032,6032,6032,6032,3.0,5,4540,NaN,NaN,NaN,NaN
6032,6033,6033,6033,6033,6033,6033,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN
6033,6034,6034,6034,6034,6034,6034,4.5,66,855,4.5,4.5,5.0,4.5
